### Imports

In [ ]:
from sklearn.datasets import load_boston
from sklearn.cluster import KMeans
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mutual_info_score
from scipy.sparse.linalg import eigs
from numpy.linalg import matrix_power
from joblib import Parallel, delayed
from sklearn.pipeline import Pipeline
from scipy.sparse.linalg import eigs
from numpy.linalg import matrix_power
from joblib import Parallel, delayed

In [ ]:
def jaccard(x,y):
    return np.mean(x!=y)

def weighted_jaccard(x,y,w=None):
    """w is list of weights, same length as x and y summing to 1"""
    if w is None:
        return jaccard(x,y)
    return (x!=y).dot(w)

def jaccard_distance_matrix(X, n_jobs=1):
    vint = np.vectorize(int)
    X_int = vint(X*100)
    return pairwise_distances(X_int, metric=jaccard, n_jobs=n_jobs)

def weighted_jaccard_distance_matrix(X, w, n_jobs=1):
    """w has length X.shape[1]"""
    vint = np.vectorize(int)
    X_int = vint(X*100)
    distance_matrix = pairwise_distances(X_int, w=w, metric=weighted_jaccard,n_jobs=n_jobs)
    return distance_matrix

class JKMedoids(object):
    def __init__(self, k, max_iter=100, n_attempts=10, accepting_weights=True, weight_adjustment=0, n_jobs=1):
        self.k = k
        self.n_attempts = n_attempts
        self.n_jobs=n_jobs
        if max_iter is None:
            self.max_iter = -1
        else:
            self.max_iter = max_iter
        self.accepting_weights = accepting_weights
        self.weight_adjustment = weight_adjustment
        self.adjusting_weights = self.weight_adjustment != 0
        self.distance_matrix = None
        self.assignments = None
        self.assignment_score = None
        self.weights = None

    def fit_once(self, X):
        assignments = np.random.randint(0,self.k,size=X.shape[0])
        old_assignments = np.zeros(assignments.shape)
        it = 0
        while (old_assignments != assignments).any() and it != self.max_iter:
            it += 1
            old_assignments = assignments

            centroids = []
            for cluster in range(self.k):
                mask = assignments == cluster
                if np.sum(mask) == 0:
                    continue
                within_cluster_distance_matrix = (self.distance_matrix[mask]).T
                most_central_point = np.argmin(np.sum(within_cluster_distance_matrix,1))
                centroids.append(most_central_point)

            to_centroid_distance_matrix = (self.distance_matrix[centroids]).T
            assignments = np.apply_along_axis(np.argmin, 1, to_centroid_distance_matrix)

            if self.adjusting_weights:
                weight_update = np.apply_along_axis(lambda col: mutual_info_score(assignments, col), 0, X)
                weight_update = weight_update/np.sum(weight_update)
                self.weights = self.weights*(1-self.weight_adjustment) + weight_update*self.weight_adjustment
                self.distance_matrix = weighted_jaccard_distance_matrix(X, self.weights)
        return assignments

    def score(self, assignments):
        centroids = []
        for cluster in range(self.k):
            mask = assignments == cluster
            if np.sum(mask) == 0:
                continue
            within_cluster_distance_matrix = (self.distance_matrix[mask]).T
            most_central_point = np.argmin(np.sum(within_cluster_distance_matrix,1))
            centroids.append(most_central_point)
        to_centroid_distance_matrix = (self.distance_matrix[centroids]).T
        scores = np.apply_along_axis(np.min, 1, to_centroid_distance_matrix)
        score = np.sum(scores)
        return score

    def fit(self, X):
        if self.accepting_weights:
            X, self.weights = X
            self.distance_matrix = weighted_jaccard_distance_matrix(X, self.weights, self.n_jobs)
        else:
            self.distance_matrix = jaccard_distance_matrix(X, self.n_jobs)
        for _ in range(self.n_attempts):
            assignments = self.fit_once(X)
            if self.assignments is None:
                self.assignments = assignments
                self.assignment_score = self.score(self.assignments)
            else:
                score = self.score(assignments)
                if score < self.assignment_score:
                    self.assignment_score = score
                    self.assignments = assignments
        return self

    def fit_predict(self, X, _=None):
        self.fit(X)
        return self.assignments

class SquishyJKMedoids(object):
    def __init__(self, k, max_iter=100, n_attempts=10, accepting_weights=True, weight_adjustment=0, n_jobs=1):
        self.k = k
        self.n_attempts = n_attempts
        if max_iter is None:
            self.max_iter = -1
        else:
            self.max_iter = max_iter
        self.accepting_weights = accepting_weights
        self.distance_matrix = None
        self.to_centroid_distances = None
        self.assignments = None
        self.assignment_score = None
        self.n_jobs=n_jobs

    def fit_once(self, X):
        assignments = np.random.randint(0,self.k,size=X.shape[0])
        old_assignments = np.zeros(assignments.shape)
        it = 0
        while (old_assignments != assignments).any() and it != self.max_iter:
            it += 1
            old_assignments = assignments

            self.to_centroid_distances = []
            centroids = []
            first_run = True
            for cluster in range(self.k):
                mask = assignments == cluster
                if np.sum(mask) == 0:
                    continue
                if first_run:
                    distance_matrix = self.distance_matrix
                else:
                    weights = np.apply_along_axis(lambda col: mutual_info_score(mask, col), 0, X)
                    weights = weights/np.sum(weights)
                    distance_matrix = weighted_jaccard_distance_matrix(X, weights, n_jobs=self.n_jobs)
                within_cluster_distance_matrix = (distance_matrix[mask]).T
                most_central_point = np.argmin(np.sum(within_cluster_distance_matrix,1))
                centroids.append(most_central_point)
                self.to_centroid_distances.append(distance_matrix[:,most_central_point].reshape(-1))

            to_centroid_distance_matrix = (np.array(self.to_centroid_distances)).T
            assignments = np.apply_along_axis(np.argmin, 1, to_centroid_distance_matrix)

            first_run = False
        return assignments

    def score(self, assignments):
        centroids = []
        for cluster in range(self.k):
            mask = assignments == cluster
            if np.sum(mask) == 0:
                continue
            within_cluster_distance_matrix = (self.distance_matrix[mask]).T
            most_central_point = np.argmin(np.sum(within_cluster_distance_matrix,1))
            centroids.append(most_central_point)
        to_centroid_distance_matrix = (self.distance_matrix[centroids]).T
        scores = np.apply_along_axis(np.min, 1, to_centroid_distance_matrix)
        score = np.sum(scores)
        return score

    def fit(self, X):
        if self.accepting_weights:
            X, self.weights = X
        else:
            self.weights = np.ones(X.shape[1])/X.shape[1]
        self.distance_matrix = weighted_jaccard_distance_matrix(X, self.weights, n_jobs=self.n_jobs)
        for _ in range(self.n_attempts):
            assignments = self.fit_once(X)
            if self.assignments is None:
                self.assignments = assignments
                self.assignment_score = self.score(self.assignments)
            else:
                score = self.score(assignments)
                if score < self.assignment_score:
                    self.assignment_score = score
                    self.assignments = assignments
        return self

    def fit_predict(self, X, _=None):
        self.fit(X)
        return self.assignments



class EigenvectorWeighting(object):
    def __init__(self, complete=False, extent=1):
        self.data = None
        self.weights = None
        self.complete = complete
        self.extent = extent

    def fit(self, data):
        data, weights = data
        self.data = data
        mutual_info_matrix = pairwise_distances(data.T, metric=mutual_info_score)
        if self.complete:
            evals, evecs = eigs(mutual_info_matrix, k=1)
            weights = evecs.reshape(-1)
            self.weights = weights/np.sum(weights)
        else:
            weights = weights.reshape(-1,1)
            mutual_info_power = matrix_power(mutual_info_matrix, self.extent)
            weights = mutual_info_power.dot(weights)
            weights = weights.reshape(-1)
            self.weights = weights/np.sum(weights)

    def transform(self, data, _=None):
        self.fit(data)
        return self.data, self.weights

    def fit_transform(self, data, _=None):
        self.fit(data)
        return self.data, self.weights
    

### Helper functions to be parallelized, if possible
def fit_rf_model(rftransform, X, i, features_to_predict):
    y_temp = X[:, features_to_predict]
    if rftransform.model_type == 'gradient_boosting':
        y_temp = np.apply_along_axis(np.mean,1,y_temp)
    X_temp = np.delete(X, features_to_predict, axis=1)
    rf_fit = rftransform.rfs[i].fit(X_temp, y_temp)
    return rf_fit

def get_predictions(rftransform, X, i, features_to_predict):
    y_temp = X[:, features_to_predict]
    X_temp = np.delete(X, features_to_predict, axis=1)
    predictions = rftransform.rfs[i].predict(X_temp)
    if len(predictions.shape) > 1:
        predictions = np.sum(predictions, 1)
    return predictions

def get_weight(rftransform, X, features_to_predict):
    y_temp = X[:, features_to_predict]
    y_temp_var = np.sum(np.apply_along_axis(np.var, 0, y_temp))
    weight = (1/y_temp_var)**rftransform.weight_extent
    return weight

class RFTransform(object):
    def __init__(self,
                 n_forests,
                 model_type='random_forest',
                 n_trees=1,
                 n_features_to_predict=0.5,
                 max_depth=5,
                 outputting_weights=True,
                 using_pca=True,
                 weight_extent=1,
                 learning_rate=0.9,
                 n_jobs=1):
        self.n_forests = n_forests
        self.n_trees = n_trees
        self.n_features_to_predict = n_features_to_predict
        self.outputting_weights = outputting_weights
        self.weight_extent = weight_extent
        self.n_jobs = n_jobs
        self.model_type = model_type
        if self.model_type == 'random_forest':
            self.rfs = [RandomForestRegressor(n_trees, max_depth=max_depth, n_jobs=-1) for _ in range(n_forests)]
        elif self.model_type == 'gradient_boosting':
            self.rfs = [GradientBoostingRegressor(n_estimators=n_trees, learning_rate=learning_rate, max_depth=max_depth) for _ in range(n_forests)]
        else:
            raise ValueError("RFTransform.model_type must be 'random_forest' or 'gradient_boosting'.")
        self.using_pca = using_pca
        if not self.using_pca and self.outputting_weights:
            print ('warning')
        self.pca = PCA()
        self.ss1 = StandardScaler()
        self.decision_paths = None
        self.features_indices = []
        if outputting_weights:
            self.weights = []

    def fit(self, X_init, *args, **kwargs):
        self.features_indices = []

        X_ss = self.ss1.fit_transform(X_init)
        if self.using_pca:
            X = self.pca.fit_transform(X_ss)
        else:
            X = X_ss
        if isinstance(self.n_features_to_predict, float):
            n_output = int(self.n_features_to_predict * X.shape[1])
        elif isinstance(self.n_features_to_predict, int):
            n_output = self.n_features_to_predict
        elif self.n_features_to_predict == 'sqrt':
            n_output = int(np.sqrt(X.shape[1]))
        elif self.n_features_to_predict == 'log':
            n_output = int(np.log2(X.shape[1]))

        if n_output == 0:
            n_output = 1

        for i in range(self.n_forests):
            features_to_predict = np.random.choice(np.arange(X.shape[1]),(n_output,),replace=False)
            self.features_indices.append(features_to_predict)

        self.rfs = Parallel(n_jobs=self.n_jobs)(delayed(fit_rf_model)(self, X, i, features_to_predict) for i, features_to_predict in enumerate(self.features_indices))
        return self

    def transform(self, X_init):
        self.decision_paths = None
        if self.outputting_weights:
            self.weights = []

        X_ss = self.ss1.transform(X_init)
        if self.using_pca:
            X = self.pca.transform(X_ss)
        else:
            X = X_ss

        decision_paths = Parallel(n_jobs=self.n_jobs)(delayed(get_predictions)(self,X,i,features_to_predict)
                                                      for i, features_to_predict in enumerate(self.features_indices))
        self.decision_paths = (np.array(decision_paths)).T

        if self.outputting_weights:
            self.weights = Parallel(n_jobs=self.n_jobs)(delayed(get_weight)(self,X,features_to_predict) for features_to_predict in self.features_indices)
            self.weights = np.array(self.weights)
            self.weights = self.weights/np.sum(self.weights)
            return self.decision_paths, self.weights
        else:
            return self.decision_paths

    def fit_transform(self, X_init, *args, **kwargs):
        self.fit(X_init)
        return self.transform(X_init)
    
import numpy as np

class RFCluster(Pipeline):
    """
    A clustering algorithm wherein a forest of shallow trees is trained on random subsets of the features (each tree trained
    on a different random subset). Points are clustered together according to how often they end up on the same leaf.
    
    Parameters:
    
      k: Int. Number of clusters to partition the data into. (Note: for large k, RFCluster often returns fewer than k clusters.
            This occurs in the final k-medoids step. If, at any given step, one data point is selected as the next centroid for
            two clusters, this reduces the number of clusters returned.
      
      n_trees: Int. Number of decision trees in the ensemble.
      
      n_features_to_predict: Float, int, or string. Determines how many features form the target for each decision tree.
            If float, takes that fraction of the total number of features (i.e., if there are 20 features in
            the data, and n_features_to_predict = 0.4, each decision tree will be trained to predict the
            values of 8 features, given the remaining 12.
            If int, that many features will be predicted with each decision tree.
            If 'sqrt', n_features_to_predict will take on the square root of the total number of features.
            If 'log', same as for 'sqrt', but using the log base 2 instead.
      
      max_depth: Int. The maximum depth of each decision tree.
      
      using_weights: Boolean. If True, when ensembling the partitions from each decision tree, some trees will have extra
            weight, given the variance of the features that the decision trees are trained on.
      
      weight_extent: Nonnegative float. Only relevant if using_weights.
      
      max_iter: Int. Maximum number of iterations in the k-medoids step.
      
      n_attempts: Int. Number of attempts for k-medoids. After n_attempts attempts, the partition with the lowest within-
            cluster-sum-of-squares is selected.
      
      k_medoids_type: String. If "normal," standard k-medoids is employed. If "minkowski", Minkowski weighted k-medoids is
            used. (This is unrelated to the Minkowski distance metric).
      
      weight_adjustment: Float. Only relevant if using_weights. The extent to which weights can be adjusted during the
            k-medoids.
      
      eig_extent: Nonnegative int. Only relevant if using_weights. Partitions gain more weight when they have more mutual
            information with other partitions. As eig_extent goes to infinity, this becomes the only criterion.
                  
      using_pca: Boolean. If True, PCA is applied to the data first.
    
    Methods:
    
    fit_predict:
        Input: 2D numpy array, each row a data point, each column a feature
        Output: 1D numpy array with cluster assignments for each data point
      
    """
    def __init__(self, k,
                n_trees=150,
                n_features_to_predict=0.5,
                max_depth=5,
                using_weights=False,
                weight_extent=1,
                max_iter=60,
                n_attempts=10,
                kmedoids_type='normal',
                weight_adjustment=0,
                eig_extent=0,
                using_pca=False,
                n_jobs=1):
        rft = RFTransform(n_trees,
                        n_features_to_predict=n_features_to_predict,
                        max_depth=max_depth,
                        outputting_weights=using_weights,
                        using_pca=using_pca,
                        weight_extent=weight_extent,
                        n_jobs=n_jobs)
        ew = EigenvectorWeighting(extent=eig_extent)
        if kmedoids_type == 'normal':
            jk = JKMedoids(k,
                            max_iter=max_iter,
                            n_attempts=n_attempts,
                            accepting_weights=using_weights,
                            weight_adjustment=weight_adjustment,
                            n_jobs=n_jobs)
        else:
            jk = SquishyJKMedoids(k,
                            max_iter=max_iter,
                            n_attempts=n_attempts,
                            accepting_weights=using_weights,
                            weight_adjustment=weight_adjustment,
                            n_jobs=n_jobs)
        if eig_extent == 0 or not using_weights:
            Pipeline.__init__(self,[('rft', rft), ('jkmeans', jk)])
        else:
            Pipeline.__init__(self,[('rft', rft), ('ew', ew), ('jkmeans', jk)])

### Main

In [ ]:
X = load_boston().data
rfc = RFCluster(k=5)
clusters = rfc.fit_predict(X)

In [ ]:
clusters